In [27]:
from sklearn.linear_model import SGDClassifier
import numpy as np

## Client Update


In [28]:
def client_update(init_weights, epochs, batch_size, features, labels):
    """
    Given the previous weights from the server, it does updates on the model
    and returns the new set of weights

    init_weights: weights to initialize the training with 
        ex: [coef1, coef2, ..., coefn, intercept]
    epochs: number of epochs to run the training for 
    batch_size: the size of each batch of data while training 
    features: a 2D array containing features for each sample 
        ex: [[feature1, feature2], [feature1, feature2], ...]
    label: an array containing the labels for the corresponding sample in "features"
        ex: [label1, label2, ...]
    """
    
    # split the data into batches by given batch_size
    # TODO: need to ensure that a batch doesn't just contain 1 label
    batches_features = []
    batches_labels = []
    
    for i in range(0, len(features), batch_size):
      batches_features.append(features[i:i + batch_size])
      batches_labels.append(labels[i:i + batch_size])
      # TODO: merge the features and labels into one tuple?
    
    print("Features: ", batches_features)
    print("Labels: ", batches_labels)

    weights = list(init_weights)
    
    # set max_iter to 1 so that each .fit() call only does one training step
    classifier = SGDClassifier(loss="hinge", penalty="l2", max_iter=1)

    for epoch in range(epochs):
        for i in range(len(batches_features)):
            print(batches_features[i], batches_labels[i], weights)
            
            classifier.fit(batches_features[i],
                           batches_labels[i],
                           coef_init=weights[:-1],
                           intercept_init=weights[-1])
            
            # update the weights so for the next batch the new ones are used
            weights = np.append(classifier.coef_[0], classifier.intercept_)
            
    return weights

# Example usage

# this data will be provided by the server
features = [[1, 4, 3], [0, 2, 2], [1, 4, 0], [0, 5, 3], [1, 2, 1], [0, 2, 9]]
labels = [1, 0, 1, 0, 1, 0]
weights = [29., 0., 0., -9]
epochs = 3
batch_size = 3

new_weights = client_update(weights, epochs, batch_size, features, labels)
print(new_weights)

Features:  [[[1, 4, 3], [0, 2, 2], [1, 4, 0]], [[0, 5, 3], [1, 2, 1], [0, 2, 9]]]
Labels:  [[1, 0, 1], [0, 1, 0]]
[[1, 4, 3], [0, 2, 2], [1, 4, 0]] [1, 0, 1] [29.0, 0.0, 0.0, -9]
[[0, 5, 3], [1, 2, 1], [0, 2, 9]] [0, 1, 0] [28.91317365  0.          0.         -9.        ]
[[1, 4, 3], [0, 2, 2], [1, 4, 0]] [1, 0, 1] [28.82660726  0.          0.         -9.        ]
[[0, 5, 3], [1, 2, 1], [0, 2, 9]] [0, 1, 0] [28.74030006  0.          0.         -9.        ]
[[1, 4, 3], [0, 2, 2], [1, 4, 0]] [1, 0, 1] [28.65425125  0.          0.         -9.        ]
[[0, 5, 3], [1, 2, 1], [0, 2, 9]] [0, 1, 0] [28.56846008  0.          0.         -9.        ]
[28.48292577  0.          0.         -9.        ]


/Users/yuxuanzhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/yuxuanzhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/yuxuanzhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/yuxuanzhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceW

## Server Update

In [29]:
from sklearn.linear_model import SGDClassifier
import random
import numpy as np

def append(list, element):
    """
    helper function to append array into array in numpy
    """
    return np.concatenate((list, [element])) if list is not None else [element]

def server(init_weight,client_num,client_fraction,round_num,features,labels,epoch,batch_size):
    
    """
    Calls clientUpdate to get the updated weights from clients, and applies Federated
    Averaging Algorithm to update the weight on server side
    
    """

    # initialize the weight
    w = init_weight
    # number of clients
    K = client_num
    # fraction of clients
    C = client_fraction

    # use to generate n_k so that the sum of n_k equals to n
    for i in range(round_num):
        # calculate the number of clients used in this round
        m = max(int(K*C),1)
        # random set of m clients
        S = np.array(random.sample(range(clients), clients))
        
        num_samples = []
            
        # grab all the weights from clients
        client_weights = None   
        for i in S:
            client_feature = features[i]
            client_label = labels[i]
            client_weights = append(client_weights,client_update(w,epoch,batch_size,client_feature,client_label))         
            num_samples.append(len(client_feature))

        # calculate the new server weight based on new weights coming from client
        new_w = np.zeros(4)
        for i in range(len(client_weights)):
            current_weight = client_weights[i]
            n_k = len(features[i])
            added_w = [value*(n_k)/sum(num_samples) for value in current_weight]
            new_w = np.add(new_w,added_w)
        
        # update the server weight to newly calculated weight
        w = new_w
        print(w)
        


In [30]:
import numpy as np

# load data

clients = 10
data_number = 30
features = np.random.randint(10,size=(clients,data_number,3))
labels = np.random.randint(2,size=(100,30))
epoch = 10
batch_size = 20

server([0,0,0,0],10,0.5,10,features,labels,epoch,batch_size)

Features:  [array([[7, 6, 4],
       [2, 5, 9],
       [2, 6, 0],
       [0, 3, 8],
       [0, 9, 4],
       [6, 3, 7],
       [8, 8, 9],
       [1, 4, 8],
       [1, 8, 9],
       [9, 2, 8],
       [9, 1, 2],
       [5, 6, 3],
       [3, 9, 3],
       [7, 9, 8],
       [4, 8, 2],
       [8, 9, 5],
       [8, 4, 9],
       [7, 6, 0],
       [5, 4, 5],
       [6, 9, 8]]), array([[2, 9, 5],
       [0, 4, 4],
       [4, 8, 8],
       [8, 3, 4],
       [5, 9, 5],
       [8, 3, 8],
       [0, 9, 7],
       [5, 0, 3],
       [8, 0, 8],
       [1, 0, 4]])]
Labels:  [array([1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0]), array([1, 0, 1, 0, 1, 0, 1, 1, 0, 0])]
[[7 6 4]
 [2 5 9]
 [2 6 0]
 [0 3 8]
 [0 9 4]
 [6 3 7]
 [8 8 9]
 [1 4 8]
 [1 8 9]
 [9 2 8]
 [9 1 2]
 [5 6 3]
 [3 9 3]
 [7 9 8]
 [4 8 2]
 [8 9 5]
 [8 4 9]
 [7 6 0]
 [5 4 5]
 [6 9 8]] [1 1 0 0 1 1 0 1 1 0 1 0 1 1 1 1 0 0 1 0] [0, 0, 0, 0]
[[2 9 5]
 [0 4 4]
 [4 8 8]
 [8 3 4]
 [5 9 5]
 [8 3 8]
 [0 9 7]
 [5 0 3]
 [8 0 8]
 [1 0 4]]

/Users/yuxuanzhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/yuxuanzhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/yuxuanzhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/yuxuanzhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceW

 [ 43.80223437  19.31335425 -58.92930022  29.9716917 ]
[[1 2 5]
 [4 4 6]
 [3 6 8]
 [5 0 8]
 [4 5 6]
 [5 2 8]
 [5 3 1]
 [0 9 5]
 [7 9 7]
 [8 3 2]] [1 0 1 1 0 1 0 0 1 1] [ 23.3154388   18.9342894  -97.02686057  39.93247082]
[[3 1 4]
 [6 1 5]
 [6 2 6]
 [7 0 4]
 [0 9 4]
 [6 7 9]
 [1 5 9]
 [4 2 1]
 [8 7 7]
 [5 9 9]
 [0 1 1]
 [5 5 4]
 [3 7 3]
 [4 7 3]
 [8 9 8]
 [1 5 0]
 [1 0 9]
 [0 5 4]
 [2 0 8]
 [1 4 3]] [1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 1 0 1 0 0] [ -6.64804424 -10.98577293 -36.60042984  40.04166629]
[[1 2 5]
 [4 4 6]
 [3 6 8]
 [5 0 8]
 [4 5 6]
 [5 2 8]
 [5 3 1]
 [0 9 5]
 [7 9 7]
 [8 3 2]] [1 0 1 1 0 1 0 0 1 1] [ 52.36369363  18.67047384 -55.50915546  40.04351231]
[[3 1 4]
 [6 1 5]
 [6 2 6]
 [7 0 4]
 [0 9 4]
 [6 7 9]
 [1 5 9]
 [4 2 1]
 [8 7 7]
 [5 9 9]
 [0 1 1]
 [5 5 4]
 [3 7 3]
 [4 7 3]
 [8 9 8]
 [1 5 0]
 [1 0 9]
 [0 5 4]
 [2 0 8]
 [1 4 3]] [1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 1 0 1 0 0] [ 12.20151629 -90.53339607 -25.22660684  30.14285752]
[[1 2 5]
 [4 4 6]
 [3 6 8]
 [5 0 8]
 [4 5 6]
 [5 2 8]
 [5

/Users/yuxuanzhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/yuxuanzhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/yuxuanzhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/yuxuanzhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceW

 [1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 1 0 1 0 0] [126.88271256 -30.55174656 -15.15913097  30.12344732]
[[1 2 5]
 [4 4 6]
 [3 6 8]
 [5 0 8]
 [4 5 6]
 [5 2 8]
 [5 3 1]
 [0 9 5]
 [7 9 7]
 [8 3 2]] [1 0 1 1 0 1 0 0 1 1] [114.57883204 -20.13856214 -54.11577217  40.1628933 ]
[[3 1 4]
 [6 1 5]
 [6 2 6]
 [7 0 4]
 [0 9 4]
 [6 7 9]
 [1 5 9]
 [4 2 1]
 [8 7 7]
 [5 9 9]
 [0 1 1]
 [5 5 4]
 [3 7 3]
 [4 7 3]
 [8 9 8]
 [1 5 0]
 [1 0 9]
 [0 5 4]
 [2 0 8]
 [1 4 3]] [1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 1 0 1 0 0] [ 24.24603886 -59.58218392 -53.57944143  30.22245698]
[[1 2 5]
 [4 4 6]
 [3 6 8]
 [5 0 8]
 [4 5 6]
 [5 2 8]
 [5 3 1]
 [0 9 5]
 [7 9 7]
 [8 3 2]] [1 0 1 1 0 1 0 0 1 1] [102.27850129  49.53621027 -42.71429047  60.1133167 ]
Features:  [array([[4, 6, 9],
       [5, 6, 4],
       [1, 9, 1],
       [6, 4, 6],
       [9, 3, 2],
       [5, 7, 9],
       [4, 7, 1],
       [6, 9, 9],
       [9, 8, 3],
       [0, 5, 1],
       [1, 9, 7],
       [3, 6, 2],
       [4, 3, 8],
       [3, 3, 9],
       [9, 6, 5],
       [0,

...

/Users/yuxuanzhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/yuxuanzhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/yuxuanzhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/yuxuanzhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceW

 [1 0 1 1 0 1 0 0 1 1] [ 96.47792213   3.87135857 -41.50992204  20.72264583]
[[3 1 4]
 [6 1 5]
 [6 2 6]
 [7 0 4]
 [0 9 4]
 [6 7 9]
 [1 5 9]
 [4 2 1]
 [8 7 7]
 [5 9 9]
 [0 1 1]
 [5 5 4]
 [3 7 3]
 [4 7 3]
 [8 9 8]
 [1 5 0]
 [1 0 9]
 [0 5 4]
 [2 0 8]
 [1 4 3]] [1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 1 0 1 0 0] [135.16495957 -95.27503745   8.45548849  10.76228937]
[[1 2 5]
 [4 4 6]
 [3 6 8]
 [5 0 8]
 [4 5 6]
 [5 2 8]
 [5 3 1]
 [0 9 5]
 [7 9 7]
 [8 3 2]] [1 0 1 1 0 1 0 0 1 1] [152.13915075 -14.89672464  -1.5240108   30.60438134]
[[3 1 4]
 [6 1 5]
 [6 2 6]
 [7 0 4]
 [0 9 4]
 [6 7 9]
 [1 5 9]
 [4 2 1]
 [8 7 7]
 [5 9 9]
 [0 1 1]
 [5 5 4]
 [3 7 3]
 [4 7 3]
 [8 9 8]
 [1 5 0]
 [1 0 9]
 [0 5 4]
 [2 0 8]
 [1 4 3]] [1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 1 0 1 0 0] [ 51.52330188 -74.21390279 -51.0629205   10.72339031]
[[1 2 5]
 [4 4 6]
 [3 6 8]
 [5 0 8]
 [4 5 6]
 [5 2 8]
 [5 3 1]
 [0 9 5]
 [7 9 7]
 [8 3 2]] [1 0 1 1 0 1 0 0 1 1] [  89.76622039  -62.9437575  -187.45030185   10.94874902]
[[3 1 4]
 [6 1 5]
 [6 2 6]
 [7 

/Users/yuxuanzhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/yuxuanzhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/yuxuanzhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/yuxuanzhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceW


[[2 9 5]
 [0 4 4]
 [4 8 8]
 [8 3 4]
 [5 9 5]
 [8 3 8]
 [0 9 7]
 [5 0 3]
 [8 0 8]
 [1 0 4]] [1 0 1 0 1 0 1 1 0 0] [ -2.50844113 123.9994428   47.82701076  12.97727284]
[[7 6 4]
 [2 5 9]
 [2 6 0]
 [0 3 8]
 [0 9 4]
 [6 3 7]
 [8 8 9]
 [1 4 8]
 [1 8 9]
 [9 2 8]
 [9 1 2]
 [5 6 3]
 [3 9 3]
 [7 9 8]
 [4 8 2]
 [8 9 5]
 [8 4 9]
 [7 6 0]
 [5 4 5]
 [6 9 8]] [1 1 0 0 1 1 0 1 1 0 1 0 1 1 1 1 0 0 1 0] [-81.77000267  53.39488935 -71.57662662  -6.92330349]
[[2 9 5]
 [0 4 4]
 [4 8 8]
 [8 3 4]
 [5 9 5]
 [8 3 8]
 [0 9 7]
 [5 0 3]
 [8 0 8]
 [1 0 4]] [1 0 1 0 1 0 1 1 0 0] [-21.28383971  81.78753137  47.59072621  -6.9717273 ]
[[7 6 4]
 [2 5 9]
 [2 6 0]
 [0 3 8]
 [0 9 4]
 [6 3 7]
 [8 8 9]
 [1 4 8]
 [1 8 9]
 [9 2 8]
 [9 1 2]
 [5 6 3]
 [3 9 3]
 [7 9 8]
 [4 8 2]
 [8 9 5]
 [8 4 9]
 [7 6 0]
 [5 4 5]
 [6 9 8]] [1 1 0 0 1 1 0 1 1 0 1 0 1 1 1 1 0 0 1 0] [-50.8053081  100.79855682  66.94066946  -7.06068703]
[[2 9 5]
 [0 4 4]
 [4 8 8]
 [8 3 4]
 [5 9 5]
 [8 3 8]
 [0 9 7]
 [5 0 3]
 [8 0 8]
 [1 0 4]] [1 0 1 0 1 0 1 1 0 0

/Users/yuxuanzhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/yuxuanzhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/yuxuanzhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/yuxuanzhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceW